<a href="https://colab.research.google.com/github/GVSU-CIS635/term-project-proposal-d-miner/blob/main/D_Miners.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [26]:
pip install xlrd

In [27]:
pip install pingouin

In [28]:
from operator import mod
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split, GridSearchCV
import csv
import pingouin as pg
from sklearn.preprocessing import LabelEncoder, StandardScaler, OneHotEncoder
from sklearn.ensemble import RandomForestClassifier
from imblearn.over_sampling import SMOTE
import matplotlib.pyplot as plt
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report, roc_curve, roc_auc_score, precision_recall_curve, f1_score, auc
from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB

In [30]:


from re import X
CSVData = pd.read_csv('https://www.eia.gov/consumption/residential/data/2020/csv/recs2020_public_v7.csv')
CSVData = pd.DataFrame(CSVData)
feature_grid = pd.read_excel('https://www.eia.gov/consumption/residential/data/2020/xls/RECS%202020%20Codebook%20for%20Public%20File%20-%20v7.xlsx', header=1)
feature_grid = pd.DataFrame(feature_grid)



In [31]:
#

main_features= CSVData.columns
print(len(main_features))

none_values = [0,-2]


799


In [32]:

selected_sections = ['GEOGRAPHY', 'ADMIN']

Y = 'KWHbins'

CSVData[Y] = pd.qcut(CSVData['KWH'], q=2, labels=[0, 1])
CSVData[Y] = CSVData[Y].astype(int)

display(feature_grid)

features = feature_grid[feature_grid['Section'].isin(selected_sections)]['Variable'].tolist()


display(features)

,Variable,Type,Description and Labels,Response Codes,Section
0,DOEID,Num,Unique identifier for each respondent,100001-118496,ADMIN
1,REGIONC,Char,Census Region,Midwest\nNortheast\nSouth\nWest,GEOGRAPHY
2,DIVISION,Char,"Census Division, Mountain Division is divided ...",East North Central\nEast South Central\nMiddle...,GEOGRAPHY
3,STATE_FIPS,Char,State Federal Information Processing System Code,state_dictionary!A1,GEOGRAPHY
4,state_postal,Char,State Postal Code,state_dictionary!A1,GEOGRAPHY
...,...,...,...,...,...
794,EVCHRGDLR,Num,"EV is charged at car dealerships, not imputed","1 Yes\n0 No\n-2 Not applicable\n""."" or blank M...",ENERGY BILLS
795,EVCHRGHWY,Num,"EV is charged at highway rest stops, not imputed","1 Yes\n0 No\n-2 Not applicable\n""."" or blank M...",ENERGY BILLS
796,EVCHRGOTH,Num,EV is charged at other public charging station...,"1 Yes\n0 No\n-2 Not applicable\n""."" or blank M...",ENERGY BILLS
797,EVHOMEAMT,Num,"Portion of EV charging done at home, not imputed",1 All at home (100%)\n2 Almost all at home (80...,ENERGY BILLS


['DOEID',
 'REGIONC',
 'DIVISION',
 'STATE_FIPS',
 'state_postal',
 'state_name',
 'BA_climate',
 'IECC_climate_code',
 'UATYP10']

In [33]:
iamb_sections = ['WEATHER', 'YOUR HOME']
iamb_features = feature_grid[feature_grid['Section'].isin(iamb_sections)]['Variable'].tolist()
iamb_features.append('KWHbins')
display(iamb_features)




missing = [0,-2]

for feature in main_features:
  if len(CSVData[CSVData[feature].isin(none_values)]) > 10000 :
   main_features = main_features.drop(feature)


blanket = [var for var in iamb_features if var in main_features]
blanket.append(Y)


print(CSVData[blanket].dtypes)
display(CSVData[Y].value_counts())

['HDD65',
 'CDD65',
 'HDD30YR_PUB',
 'CDD30YR_PUB',
 'TYPEHUQ',
 'CELLAR',
 'CRAWL',
 'CONCRETE',
 'BASEOTH',
 'BASEFIN',
 'ATTIC',
 'ATTICFIN',
 'STORIES',
 'PRKGPLC1',
 'SIZEOFGARAGE',
 'KOWNRENT',
 'YEARMADERANGE',
 'BEDROOMS',
 'NCOMBATH',
 'NHAFBATH',
 'OTHROOMS',
 'TOTROOMS',
 'STUDIO',
 'WALLTYPE',
 'ROOFTYPE',
 'HIGHCEIL',
 'DOOR1SUM',
 'WINDOWS',
 'TYPEGLASS',
 'ORIGWIN',
 'WINFRAME',
 'TREESHAD',
 'ADQINSUL',
 'DRAFTY',
 'UGASHERE',
 'SWIMPOOL',
 'MONPOOL',
 'POOLPUMP',
 'FUELPOOL',
 'RECBATH',
 'MONTUB',
 'FUELTUB',
 'SQFTEST',
 'SQFTRANGE',
 'SQFTINCB',
 'SQFTINCA',
 'SQFTINCG',
 'TOTSQFT_EN',
 'TOTHSQFT',
 'TOTCSQFT',
 'ZADQINSUL',
 'ZATTIC',
 'ZATTICFIN',
 'ZBASEFIN',
 'ZBASEOTH',
 'ZBEDROOMS',
 'ZCELLAR',
 'ZCONCRETE',
 'ZCRAWL',
 'ZDOOR1SUM',
 'ZDRAFTY',
 'ZFUELPOOL',
 'ZFUELTUB',
 'ZHIGHCEIL',
 'ZKOWNRENT',
 'ZMONPOOL',
 'ZMONTUB',
 'ZNCOMBATH',
 'ZNHAFBATH',
 'ZORIGWIN',
 'ZOTHROOMS',
 'ZPOOLPUMP',
 'ZPRKGPLC1',
 'ZRECBATH',
 'ZROOFTYPE',
 'ZSIZEOFGARAGE',
 'ZSQFTEST'

HDD65              int64
CDD65              int64
HDD30YR_PUB        int64
CDD30YR_PUB        int64
TYPEHUQ            int64
ATTIC              int64
STORIES            int64
PRKGPLC1           int64
SIZEOFGARAGE       int64
KOWNRENT           int64
YEARMADERANGE      int64
BEDROOMS           int64
NCOMBATH           int64
OTHROOMS           int64
TOTROOMS           int64
WALLTYPE           int64
ROOFTYPE           int64
HIGHCEIL           int64
DOOR1SUM           int64
WINDOWS            int64
TYPEGLASS          int64
ORIGWIN            int64
WINFRAME           int64
TREESHAD           int64
ADQINSUL           int64
DRAFTY             int64
UGASHERE           int64
SQFTEST            int64
SQFTRANGE          int64
TOTSQFT_EN         int64
TOTHSQFT           int64
TOTCSQFT           int64
DBT1             float64
DBT99            float64
GWT              float64
KWHbins            int64
dtype: object


,count
KWHbins,
1,9248
0,9248


In [34]:
def iamb(data, target, alpha=0.05):

    # Initialize the Markov Blanket (MB) for the target as an empty list
    MB_Y = []

    # Initialize the set of remaining variables (excluding the target variable)
    remaining_variables = set(data.columns) - {target}

    while remaining_variables:
        max_pval = float('inf')  # Initialize the max p-value as infinity
        best_variable = None

        # Loop through the remaining variables to find the best one
        for variable in remaining_variables:
            # Compute the partial correlation between the target and the variable
            # Conditioning on the current Markov blanket (MB_Y)
            partial_corr = pg.partial_corr(data=data, x=target, y=variable, covar=MB_Y)

            # If the p-value is smaller than alpha and larger than the current max p-value
            pval = partial_corr['p-val'].values[0]
            if pval < alpha and pval < max_pval:
                max_pval = pval
                best_variable = variable

        # If a best variable was found that satisfies the significance condition
        if best_variable:
            MB_Y.append(best_variable)
            remaining_variables.remove(best_variable)
        else:
            # If no variable satisfies the p-value condition, break out of the loop
            break

    return MB_Y

In [35]:
markov_blanket = iamb(CSVData[blanket], Y)




display(len(markov_blanket))
display(markov_blanket)

display(CSVData[markov_blanket])

iamb_csvdata = CSVData[markov_blanket]

22

['NCOMBATH',
 'TYPEHUQ',
 'CDD30YR_PUB',
 'TOTCSQFT',
 'UGASHERE',
 'TOTROOMS',
 'DRAFTY',
 'SQFTRANGE',
 'TOTSQFT_EN',
 'KOWNRENT',
 'CDD65',
 'DBT1',
 'BEDROOMS',
 'GWT',
 'DBT99',
 'HDD30YR_PUB',
 'TOTHSQFT',
 'STORIES',
 'OTHROOMS',
 'TREESHAD',
 'WINFRAME',
 'TYPEGLASS']

,NCOMBATH,TYPEHUQ,CDD30YR_PUB,TOTCSQFT,UGASHERE,TOTROOMS,DRAFTY,SQFTRANGE,TOTSQFT_EN,KOWNRENT,...,BEDROOMS,GWT,DBT99,HDD30YR_PUB,TOTHSQFT,STORIES,OTHROOMS,TREESHAD,WINFRAME,TYPEGLASS
0,3,2,1027,2100,1,8,3,6,2100,1,...,4,58.5,21.6,4451,2100,1,4,1,3,2
1,1,5,1305,590,1,3,3,1,590,2,...,1,57.2,16.1,4429,590,-2,2,0,1,2
2,1,5,1010,900,1,4,4,3,900,2,...,1,58.5,21.6,4500,900,-2,3,0,2,1
3,2,2,1653,2100,1,9,4,6,2100,1,...,2,61.5,25.1,3229,2100,2,7,1,3,2
4,1,5,1059,800,1,3,3,3,800,2,...,1,55.9,13.0,4896,800,-2,2,0,2,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
18491,2,3,1190,1500,1,5,4,4,1500,1,...,3,54.9,17.9,4547,1500,2,2,0,3,2
18492,1,2,186,1530,0,5,3,5,3070,1,...,3,39.1,-8.9,9862,3070,1,2,0,1,2
18493,2,1,4047,900,1,5,4,5,1500,1,...,3,78.6,42.0,672,0,-2,2,0,1,1
18494,3,2,2295,3000,0,9,3,8,3000,1,...,3,70.4,31.5,1752,3000,2,6,1,3,2


In [36]:
categorical_features = ['REGIONC', 'DIVISION', 'BA_climate', 'UATYP10']

enc_data = CSVData[categorical_features]
display(enc_data)

enc= OneHotEncoder(dtype=np.int64)
enc_data_fit = enc.fit_transform(enc_data).toarray()
enc_data_fit = pd.DataFrame(enc_data_fit)
enc_data_fit.columns = enc.get_feature_names_out()
display(enc_data_fit)

,REGIONC,DIVISION,BA_climate,UATYP10
0,WEST,Mountain South,Mixed-Dry,U
1,SOUTH,West South Central,Mixed-Humid,U
2,WEST,Mountain South,Mixed-Dry,U
3,SOUTH,South Atlantic,Mixed-Humid,U
4,NORTHEAST,Middle Atlantic,Mixed-Humid,U
...,...,...,...,...
18491,SOUTH,South Atlantic,Mixed-Humid,U
18492,NORTHEAST,New England,Very-Cold,R
18493,SOUTH,West South Central,Hot-Humid,U
18494,SOUTH,South Atlantic,Hot-Humid,U


,REGIONC_MIDWEST,REGIONC_NORTHEAST,REGIONC_SOUTH,REGIONC_WEST,DIVISION_East North Central,DIVISION_East South Central,DIVISION_Middle Atlantic,DIVISION_Mountain North,DIVISION_Mountain South,DIVISION_New England,...,BA_climate_Hot-Dry,BA_climate_Hot-Humid,BA_climate_Marine,BA_climate_Mixed-Dry,BA_climate_Mixed-Humid,BA_climate_Subarctic,BA_climate_Very-Cold,UATYP10_C,UATYP10_R,UATYP10_U
0,0,0,0,1,0,0,0,0,1,0,...,0,0,0,1,0,0,0,0,0,1
1,0,0,1,0,0,0,0,0,0,0,...,0,0,0,0,1,0,0,0,0,1
2,0,0,0,1,0,0,0,0,1,0,...,0,0,0,1,0,0,0,0,0,1
3,0,0,1,0,0,0,0,0,0,0,...,0,0,0,0,1,0,0,0,0,1
4,0,1,0,0,0,0,1,0,0,0,...,0,0,0,0,1,0,0,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
18491,0,0,1,0,0,0,0,0,0,0,...,0,0,0,0,1,0,0,0,0,1
18492,0,1,0,0,0,0,0,0,0,1,...,0,0,0,0,0,0,1,0,1,0
18493,0,0,1,0,0,0,0,0,0,0,...,0,1,0,0,0,0,0,0,0,1
18494,0,0,1,0,0,0,0,0,0,0,...,0,1,0,0,0,0,0,0,0,1


In [37]:
features = markov_blanket

Y_data = CSVData[Y]
X_data = pd.concat([CSVData[features], enc_data_fit], axis=1)
X_train, X_test, Y_train, Y_test = train_test_split(X_data, Y_data, test_size = 0.3)


print(X_train.dtypes)

NCOMBATH                         int64
TYPEHUQ                          int64
CDD30YR_PUB                      int64
TOTCSQFT                         int64
UGASHERE                         int64
TOTROOMS                         int64
DRAFTY                           int64
SQFTRANGE                        int64
TOTSQFT_EN                       int64
KOWNRENT                         int64
CDD65                            int64
DBT1                           float64
BEDROOMS                         int64
GWT                            float64
DBT99                          float64
HDD30YR_PUB                      int64
TOTHSQFT                         int64
STORIES                          int64
OTHROOMS                         int64
TREESHAD                         int64
WINFRAME                         int64
TYPEGLASS                        int64
REGIONC_MIDWEST                  int64
REGIONC_NORTHEAST                int64
REGIONC_SOUTH                    int64
REGIONC_WEST             

In [38]:
# Decision Tree Grid Search
dt_param_grid = {
    'max_depth': [2, 3, 4, 5, 6, 7, 8],
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1, 2, 4],
    'criterion': ['gini', 'entropy']
}
model_tree = DecisionTreeClassifier(random_state=0)
grid_search = GridSearchCV(estimator = model_tree, param_grid = dt_param_grid, cv=5)
grid_search.fit(X_train, Y_train)
best_params = grid_search.best_params_
best_estimator = grid_search.best_estimator_
print(best_params)
print(best_estimator)

{'criterion': 'gini', 'max_depth': 5, 'min_samples_leaf': 4, 'min_samples_split': 2}
DecisionTreeClassifier(max_depth=5, min_samples_leaf=4, random_state=0)


In [39]:
# Logistic Regression Grid Search


lr_param_grid = {'C': [0.001, 0.01, 0.1, 1, 10, 100, 1000] }
model_log = LogisticRegression(solver='liblinear')
grid_search = GridSearchCV(estimator = model_log, param_grid = lr_param_grid, cv=5)
grid_search.fit(X_train, Y_train)
best_params = grid_search.best_params_
best_estimator = grid_search.best_estimator_
print(best_params)
print(best_estimator)

{'C': 10}
LogisticRegression(C=10, solver='liblinear')


In [40]:
# Gaussian Naive Bayes Grid Search
gnb_param_grid = {'var_smoothing': np.logspace(0,-9, num=100)}
model_nb = GaussianNB()
grid_search = GridSearchCV(estimator = model_nb, param_grid = gnb_param_grid, cv=5)
grid_search.fit(X_train, Y_train)
best_params = grid_search.best_params_
best_estimator = grid_search.best_estimator_
print(best_params)
print(best_estimator)

{'var_smoothing': 5.336699231206302e-09}
GaussianNB(var_smoothing=5.336699231206302e-09)


In [ ]:
# KNN Grid Search


knn_param_grid = {'n_neighbors': np.arange(1, 25)}
model_knn = KNeighborsClassifier()
grid_search = GridSearchCV(estimator = model_knn, param_grid = knn_param_grid, cv=5)
grid_search.fit(X_train, Y_train)
best_params = grid_search.best_params_
best_estimator = grid_search.best_estimator_

print(best_params)
print(best_estimator)

In [ ]:
model_list = {
    'KNN': model_knn,
    'Logistic Regression': model_log,
    'Decision Tree': model_tree,
    'Gaussian Naive Bayes': model_nb
}


plt.figure(figsize=(10, 6))
for model_name, model in model_list.items():
    model.fit(X_train, Y_train)
    y_pred_prob = model.predict_proba(X_test)[:, 1]
    fpr, tpr, _ = roc_curve(Y_test, y_pred_prob)
    roc_auc = auc(fpr, tpr)
    plt.plot(fpr, tpr, label=f'{model_name} (AUC = {roc_auc:.2f})')

plt.plot([0, 1], [0, 1], 'k--', label='Guessing')
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('ROC Curve')
plt.legend(loc= 'lower right')
plt.show()


In [ ]:
# Find F1-score, Accuracy and AUC-ROC
y_pred = best_estimator.predict(X_test)
f1 = f1_score(Y_test, y_pred)
accuracy = accuracy_score(Y_test, y_pred)

# Graph the AUC-ROC curve
fpr, tpr, thresholds = roc_curve(Y_test, y_pred)
roc_auc = auc(fpr, tpr)
plt.figure()
plt.plot(fpr, tpr, color='green', lw=2, label='ROC curve (area = %0.2f)' % roc_auc)
plt.legend()
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('Receiver Operating Characteristic')
plt.show()